## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-23-02-14-21 +0000,bbc,Tesla profits slide despite record revenue,https://www.bbc.com/news/articles/cql9yqxllqdo...
1,2025-10-23-02-09-06 +0000,nyt,U.S. Strikes 2nd Boat in Pacific as Anti-Drug ...,https://www.nytimes.com/2025/10/22/us/politics...
2,2025-10-23-02-05-18 +0000,48hills,"BIG WEEK: Lila Downs, 33 years of MORE!, Sandc...",https://48hills.org/2025/10/big-week-lila-down...
3,2025-10-23-02-04-46 +0000,nyt,Three Offers in One Month: Paramount’s Secret ...,https://www.nytimes.com/2025/10/22/business/de...
4,2025-10-23-02-03-15 +0000,bbc,Person of interest in British girl's disappear...,https://www.bbc.com/news/articles/cx2082pqyl2o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2469/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
156,trump,44
101,israel,16
56,new,15
51,mamdani,13
271,gaza,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
112,2025-10-22-20-58-00 +0000,wsj,President Trump has privately told allies he b...,https://www.wsj.com/politics/elections/trump-z...,131
45,2025-10-22-23-29-51 +0000,nypost,Andrew Cuomo declares Trump will knock Zohran ...,https://nypost.com/2025/10/22/us-news/andrew-c...,122
44,2025-10-22-23-30-11 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,96
28,2025-10-23-00-34-00 +0000,wsj,The White House is tearing down the entire Eas...,https://www.wsj.com/politics/policy/white-hous...,95
38,2025-10-22-23-45-17 +0000,nypost,Andrew Cuomo mocks Zohran Mamdani over anti-Is...,https://nypost.com/2025/10/22/us-news/andrew-c...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
112,131,2025-10-22-20-58-00 +0000,wsj,President Trump has privately told allies he b...,https://www.wsj.com/politics/elections/trump-z...
38,67,2025-10-22-23-45-17 +0000,nypost,Andrew Cuomo mocks Zohran Mamdani over anti-Is...,https://nypost.com/2025/10/22/us-news/andrew-c...
302,56,2025-10-22-10-00-00 +0000,nypost,Biden’s cognitive decline emboldened Putin to ...,https://nypost.com/2025/10/22/us-news/bidens-d...
253,51,2025-10-22-13-31-00 +0000,wsj,Paul Ingrassia withdrew Tuesday from a Senate ...,https://www.wsj.com/politics/policy/paul-ingra...
364,47,2025-10-22-03-11-49 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
77,46,2025-10-22-22-08-46 +0000,nypost,Elon Musk’s Tesla disappoints investors despit...,https://nypost.com/2025/10/22/business/elon-mu...
137,45,2025-10-22-20-16-45 +0000,nypost,GM embraces AI with plans to use ‘eyes-off’ dr...,https://nypost.com/2025/10/22/business/gm-embr...
325,43,2025-10-22-09-00-00 +0000,wsj,The Republican National Committee heads into t...,https://www.wsj.com/politics/policy/democratic...
150,41,2025-10-22-19-35-20 +0000,nypost,America’s most infamous jewel thief slams Louv...,https://nypost.com/2025/10/22/lifestyle/infamo...
138,40,2025-10-22-20-11-34 +0000,nypost,Top Wall Street investor warns Zohran Mamdani’...,https://nypost.com/2025/10/22/business/top-wal...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
